In [1]:
#File: dataPreprocessing.ipynb
#Purpose: data preprocessing for steve mapping dataset
#Author: Quan Gan
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/Raw data/steve_mapping_1000.csv', skiprows = 1)

In [3]:
info = df[['collectionCode', 'habitat', 'higherGeography', 'locality', 'higherClassification', 'Steve label']]

In [4]:
#recrods' labels clean
info = info.replace(np.nan, "", regex=True)
info = info[info['Steve label'] != ""]
info = info.replace('\n', " ", regex=True)
info = info.replace('Lake,', "Lake", regex=True)

In [5]:
#add prefix "__label__" to labels
labels = info['Steve label'].str.split(r', ', expand=True)
labels = labels.rename(columns= {0: "label1", 1: "label2"})
labels['label1'] = "__label__" + labels['label1']
labels = labels.fillna("")
labels.loc[labels['label2'] != "", 'label2'] = "__label__"+labels.loc[labels['label2'] != ""]['label2'] 

In [6]:
result = labels.replace(" ", "_", regex=True)

In [7]:
#records' text clean
info['text'] = info['collectionCode'] + " " + info['habitat'] + " " + info['higherGeography'] + " " + info['locality'] + " " + info['higherClassification']
info['text'] = info['text'].str.replace('[^\w\s]','')
info['text'] = info['text'].str.lower()

/var/folders/3m/_4pt01ws7ks9h8k6fp1bp2qw0000gn/T/ipykernel_42087/4128483157.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  info['text'] = info['text'].str.replace('[^\w\s]','')


In [8]:
#sample records to trainset and testset
result['label'] = result['label1'] + " " + result['label2']
result['text'] = info['text']
ds = result['label'] + " " + result['text']
train = ds.sample(frac=0.7, random_state=99) #randomly choose 70% records as trainset
test = ds.loc[~ds.index.isin(train.index)] #rest 30% records as testset

In [9]:
train.to_csv(r'../data/steve_696.train', header=False, index=False)
test.to_csv(r'../data/steve_299.valid', header=False, index=False)